# Geodata correction
First, the old geodata is uploaded and the most frequent locations are retrieved. The locations that are too specific to be true, or that could be names wrongly considered locations by the NER algorithm are written into a CSV.

Then, a function takes the incorrect locations from the CSV and drops or substitutes them.

Some volumes are lost after the correction: total 15874 random volumes.

In [0]:
%%time
#Create dataframe and print head for reference
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as scs
import os
from csv import DictReader

geo = pd.read_csv('old_geo.csv.gz')


In [0]:
#Create CSV of most frequent urban locations
urban.groupby(["text_string", "formatted_address"], as_index=False)["occurs_100k"].sum().sort_values(by="occurs_100k", ascending=False)[0:1000].to_csv(os.path.join(".", 'top_locations_0,1000.csv'))

In [0]:
#Substitute wrong locations

def substitutions_list(file_path):

    subst_list = []
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = DictReader(csvfile)
        for row in reader:
            subst_tuple = (row["wrong"], row["correct"])
            subst_list.append(subst_tuple)
            
        return subst_list


def substitution(geodata, subst_list):

    for tup in subst_list:
        geodata.loc[geodata["formatted_address"] == tup[0], ["formatted_address"]] = tup[1]
    
    return geodata


In [0]:
s_list = substitutions_list("geo_subst.csv")

In [0]:
substitution(geo, s_list)

In [0]:
#Deleting non urban locations (aka. personal names)

def corrections_list(file_path):

    corr_list = []
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = DictReader(csvfile)
        for row in reader:
            corr_list.append(row["drop"])
            
        return corr_list

def corrections_idx(geodata, corr_list):
    
    indexes = []
    for address in corr_list:
        idx = geodata.index[geodata["formatted_address"] == address].tolist()
        indexes.extend(idx)
        
    return indexes

In [0]:
c_list = corrections_list("geo_drop.csv")

In [0]:
idx_list = corrections_idx(geo, c_list)
len(idx_list)

50781

In [0]:
%%time
corr_geo = geo[np.bincount(idx_list, minlength=len(geo)) == 0]

CPU times: user 784 ms, sys: 656 ms, total: 1.44 s
Wall time: 1.46 s


In [0]:
corr_geo.to_csv('corr_geo.csv.gz', compression='gzip')

In [0]:
#Test substition
corr_geo.loc[geo["formatted_address"] == "Mars"]

In [0]:
#Test drop
corr_geo.loc[corr_geo["formatted_address"] == "Productos Santa Mónica, Av Costa de Oro 648, Centro, 91700 Veracruz, Ver., Mexico"]

,htid,text_string,corpus,occurs_raw,occurs_100k,title,author,year,wordcount,formatted_address,...,admin_1_std,admin_2,admin_3,locality,sublocality_1,neighborhood,route,colloquial_area,lat,lon
